This notebook contains code to replicate the workflow and results presented in Section 3.2 of "spectral-recovery: A Python package for spectral recovery analysis of forest ecosystems".

The workflow can be broken down into 4 steps:

1. Read timeseries data/compute indices
2. Define the restoration site
3. Compute a recovery target
4. Compute recovery metrics

In [ ]:
!pip install spectral_recovery

In [ ]:
import spectral_recovery as sr
from spectral_recovery import data

In [ ]:
# 1. Read in the timeseries data and compute indices
ts = sr.read_timeseries(
    paths=data.bc06_wildfire_landsat_bap_timeseries(),
    band_names={1: "blue", 2: "green", 3: "red", 4: "nir", 5: "swir16", 6: "swir22"},
)
indices = sr.compute_indices(ts, indices=["NBR"])

In [ ]:
# 2. Define the restoration site
rest_site = sr.read_restoration_sites(
    path=data.bc06_wildfire_restoration_site(),
    dist_rest_years={0: [2006, 2007]}
)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 5, figsize=(15, 12))
axs = axs.flatten()

# Plot the band data
rgb1 = ts.sel(band=["R", "G", "B"], time="2005-01-01")
rgb1.plot.imshow(ax=axs[0])
rest_site.to_crs(indices.spatial_ref.crs_wkt).plot(ax=axs[0], edgecolor='red', facecolor='none', linewidth=2)
axs[0].set_title("2005")

rgb1 = ts.sel(band=["R", "G", "B"], time="2006-01-01")
rgb1.plot.imshow(ax=axs[1])
rest_site.to_crs(indices.spatial_ref.crs_wkt).plot(ax=axs[1], edgecolor='red', facecolor='none', linewidth=2)
axs[1].set_title("2006")

rgb2 = ts.sel(band=["R", "G", "B"], time="2007-01-01")
rgb2.plot.imshow(ax=axs[2])
rest_site.to_crs(indices.spatial_ref.crs_wkt).plot(ax=axs[2], edgecolor='red', facecolor='none', linewidth=2)
axs[2].set_title("2007 (post-fire)")

rgb3 = ts.sel(band=["R", "G", "B"], time="2008-01-01")
rgb3.plot.imshow(ax=axs[3])
rest_site.to_crs(indices.spatial_ref.crs_wkt).plot(ax=axs[3], edgecolor='red', facecolor='none', linewidth=2)
axs[3].set_title("2008")

rgb4 = ts.sel(band=["R", "G", "B"], time="2009-01-01")
rgb4.plot.imshow(ax=axs[4])
rest_site.to_crs(indices.spatial_ref.crs_wkt).plot(ax=axs[4], edgecolor='red', facecolor='none', linewidth=2)
axs[4].set_title("2009")

In [ ]:
# 3. Compute a historic median recovery target
target = sr.targets.historic.median(
    restoration_sites=rest_site,
    timeseries_data=indices,
    reference_years={0:[2003, 2005]},
    scale="pixel"
)
target

In [ ]:
# 4. Compute R80P and Y2R recovery metrics 
metrics = sr.compute_metrics(
    metrics=["dNBR", "R80P", "YrYr", "Y2R"],
    restoration_sites=rest_site,
    timeseries_data=indices,
    recovery_targets=target, 
)
metrics

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

fig, axs = plt.subplots(1, 4, figsize=(4*4, 4*1))
axs = axs.flatten()

# Plot dIR
img = metrics[0].sel(metric="dNBR", band="NBR")
im = axs[0].imshow(img, cmap='RdYlGn', vmax="0.5", vmin="-0.5")
axs[0].set_title("dNBR (2007-2012)")
axs[0].axis('off')
plt.colorbar(im, ax=axs[0], fraction=0.046, pad=0.04)

# Plot R80P
img = metrics[0].sel(metric="R80P", band="NBR")
im = axs[1].imshow(img, cmap='YlGn', vmax="1", vmin="0")
axs[1].set_title("R80P (2007-2012)")
axs[1].axis('off')
plt.colorbar(im, ax=axs[1], fraction=0.046, pad=0.04)

# Plot YrYr
img = metrics[0].sel(metric="YrYr", band="NBR")
im = axs[2].imshow(img, cmap='RdYlGn', vmax="0.15", vmin="-0.15")
axs[2].set_title(f'YrYr (2007-2012)')
axs[2].axis('off')
plt.colorbar(im, ax=axs[2], fraction=0.046, pad=0.04)

# Plot Y2R (without unrecovered pixels)
img = metrics[0].sel(metric="Y2R", band="NBR")
img_nur = xr.where(img < 0, np.nan, img)
im = axs[3].imshow(img_nur, cmap='Greens_r')
# Plot unrecovered Y2R pixels
img_unrec = xr.where(img == -9999, 1, np.nan)
im_unrec= axs[3].imshow(img_unrec, cmap='bwr_r', interpolation='nearest')
axs[3].set_title("Y2R (2007-2024)")
axs[3].axis('off')
plt.colorbar(im, ax=axs[3], fraction=0.046, pad=0.04)

You can also write the results to file:

In [ ]:
# How to write results to TIFs? 

# Index the `metrics` variable by the polygon number (e.g 0),
# select the metric or band you want to write with `sel()`
# then use rioxarray's `to_raster` function to write:
metrics[0].sel(metric="R80P").rio.to_raster("R80P_write_test.tif")